In [26]:
# essentials
import numpy as np
import matplotlib.pyplot as plt
import torch

# loading data in
from torchvision.datasets import ImageFolder
from torchvision import transforms as T

# putting data into batches (prevent memory overload)
from torch.utils.data import DataLoader

# for creating model
from torch import nn,optim

In [67]:
# we augment the training data to reduce overfitting on validation set and 
# eventual use on the web app
# we randomly flip/rotate the training image

# we convert images to tensor objects in both training and validation

train_augs = T.Compose([
    T.Grayscale(1),
    T.RandomHorizontalFlip(p = 0.5),
    T.RandomRotation(degrees=(-15, + 15)),
    T.ToTensor()
])

valid_augs = T.Compose([
    T.Grayscale(1),
    T.ToTensor()
])

In [68]:
# loading the data in using the helpful torchvision ImageFolder lib
# (this lets us skip manually creation of the dataset)
# we apply the transforms here

train_path = "C:\\Users\\fuchsga\\Desktop\\WebApps\\FacialAnalysisWebApp\\images\\train"
vildation_path = "C:\\Users\\fuchsga\\Desktop\\WebApps\\FacialAnalysisWebApp\\images\\validation"

trainset = ImageFolder(train_path, transform = train_augs)
validset = ImageFolder(vildation_path, transform = valid_augs)

In [69]:
print(trainset.class_to_idx)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [137]:
batch_size = 64

train_loader = DataLoader(trainset, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(validset, batch_size = batch_size)

In [263]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=7, n_channel=32, drop=0.2):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(n_input, n_channel, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(drop)
            )
        self.layer2 = nn.Sequential(
            nn.Conv2d(n_channel, 2*n_channel, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(drop)
            )
        self.layer3 = nn.Sequential(
            nn.Conv2d(2*n_channel, 2*n_channel, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(drop)
            )
        self.layer4 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, 128)
        )
        self.layer5 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, n_output)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return nn.functional.log_softmax(x)

device = 'cuda'

model = M5(n_output=len(trainset.classes))
model.to(device)
print(model)

n = sum(p.numel() for p in model.parameters())
print("Number of parameters: %s" % n)

M5(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
  )
  (layer4): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1024, out_features=128, bias=True)
  )
  (layer5): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=128, out_features=7, bias=True)
  )
)
Number of parameters: 188359


In [264]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)  # reduce the learning after 20 epochs by a factor of 10

In [265]:
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)

        output = model(data)

        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = nn.functional.nll_loss(output.squeeze(), target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

In [266]:
n_epoch = 50
log_interval=20

losses = []

# The transform needs to live on the same device as the model and the data.
for epoch in range(1, n_epoch + 1):
  train(model, epoch, log_interval)
  # test(model, epoch)
  scheduler.step()

C:\Users\fuchsga\AppData\Local\Temp\ipykernel_29756\2981391630.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.log_softmax(x)


Train Epoch: 1 [0/28821 (0%)]	Loss: 1.971704
Train Epoch: 1 [1280/28821 (4%)]	Loss: 1.880917
Train Epoch: 1 [2560/28821 (9%)]	Loss: 1.883081
Train Epoch: 1 [3840/28821 (13%)]	Loss: 1.763613
Train Epoch: 1 [5120/28821 (18%)]	Loss: 1.688683
Train Epoch: 1 [6400/28821 (22%)]	Loss: 1.972862
Train Epoch: 1 [7680/28821 (27%)]	Loss: 1.703659
Train Epoch: 1 [8960/28821 (31%)]	Loss: 1.661169
Train Epoch: 1 [10240/28821 (35%)]	Loss: 1.839781
Train Epoch: 1 [11520/28821 (40%)]	Loss: 1.886461
Train Epoch: 1 [12800/28821 (44%)]	Loss: 1.755640
Train Epoch: 1 [14080/28821 (49%)]	Loss: 1.663437
Train Epoch: 1 [15360/28821 (53%)]	Loss: 1.668988
Train Epoch: 1 [16640/28821 (58%)]	Loss: 1.676291
Train Epoch: 1 [17920/28821 (62%)]	Loss: 1.770128
Train Epoch: 1 [19200/28821 (67%)]	Loss: 1.601039
Train Epoch: 1 [20480/28821 (71%)]	Loss: 1.741401
Train Epoch: 1 [21760/28821 (75%)]	Loss: 1.732704
Train Epoch: 1 [23040/28821 (80%)]	Loss: 1.768159
Train Epoch: 1 [24320/28821 (84%)]	Loss: 1.626320
Train Epoch: 1

In [269]:
tot = 0
corr = 0
model.eval()
for _, (data, target) in enumerate(valid_loader):
    data = data.to(device)
    target = target.to(device)
    pred = model(data)
    pred = pred.argmax(dim=-1)
    correct = pred == target
    corr += sum(correct)
    tot += len(data)

print(str(corr/tot*100))
    

C:\Users\fuchsga\AppData\Local\Temp\ipykernel_29756\2981391630.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.log_softmax(x)


tensor(58.4772, device='cuda:0')


In [268]:
save_path = "C:\\Users\\fuchsga\\Desktop\\WebApps\\FacialAnalysisWebApp\\model2.pt"
torch.save(model.state_dict(), save_path)

In [274]:
data[0].shape

torch.Size([1, 48, 48])